

For an up-to-date version / full view of the plotly - plots, please, go to

Data Cleaning: https://drive.google.com/file/d/19I0eztUcGgw7AdexyUdb3lCEU5AAO4vZ/view?usp=sharing

# LendingClub Prediction: Data Cleaning


## Import Modules

In [1]:
%%capture
#! pip install -q kaggle
#! pip install -q pingouin
! pip install -q fastparquet==2024.2.0
! pip install -q dask==2023.8.1

! pip install -q pandas==1.5.3
! pip install -q numpy==1.25.2


In [2]:
%%capture
! pip install -q memory_profiler

In [3]:
%load_ext memory_profiler

In [4]:
import numpy as np
import pandas as pd

import sys
import os
import warnings
from importlib import reload

from dask import dataframe as dd
#import matplotlib.pyplot as plt
#import seaborn as sns
#import plotly.express as px

#from statsmodels.stats.proportion import proportions_ztest
#import pingouin as pg



from google.colab import drive
drive.mount("/content/gdrive")

warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
#pd.reset_option('display.max_rows')

Mounted at /content/gdrive


In [5]:
home_folder = '/content/gdrive/MyDrive/Colab Notebooks/Portfolio/ML_LendingClub/'

### Functions

In [8]:
#import dask.typing as dasktype


In [9]:
#dir(dasktype)

['Any',
 'Callable',
 'CollType',
 'CollType_co',
 'DaskCollection',
 'HLGDaskCollection',
 'Hashable',
 'Mapping',
 'PostComputeCallable',
 'PostPersistCallable',
 'Protocol',
 'SchedulerGetCallable',
 'Sequence',
 'TYPE_CHECKING',
 'TypeVar',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'abc',
 'annotations',
 'runtime_checkable']

The Python-file with the functions is at
https://drive.google.com/file/d/1JWLo8HAhUdu5xkDG4-tu7MiUSLgSxOjQ/view?usp=sharing

In [18]:
%load_ext autoreload
%autoreload 2

sys.path.append(home_folder)
import lendfunc as lfunc


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Load and Update Data

source : https://storage.googleapis.com/335-lending-club/lending-club.zip

In [19]:


dtype={'id': 'object',
       'policy_code': 'object',
       'Policy Code': 'object',
       'sec_app_earliest_cr_line': 'object',
       'debt_settlement_flag_date': 'object',
       'desc': 'object',
       'hardship_end_date': 'object',
       'hardship_loan_status': 'object',
       'hardship_reason': 'object',
       'hardship_start_date': 'object',
       'hardship_status': 'object',
       'hardship_type': 'object',
       'payment_plan_start_date': 'object',
       'settlement_date': 'object',
       'settlement_status': 'object'}

csv_accepted = home_folder+'original_data/accepted_2007_to_2018Q4.csv'
csv_rejected = home_folder+'original_data/rejected_2007_to_2018Q4.csv'

LCacc_dd = dd.read_csv(csv_accepted, dtype=dtype)
LCrej_dd = dd.read_csv(csv_rejected, dtype=dtype)


In [20]:
#LCacc_dd.dtypes

In [21]:
cat_acc_feat = LCacc_dd.select_dtypes(include='object').columns.tolist()
cat_rej_feat = LCrej_dd.select_dtypes(include='object').columns.tolist()
num_acc_feat = LCacc_dd.select_dtypes(include=np.number).columns.tolist()
num_rej_feat = LCrej_dd.select_dtypes(include=np.number).columns.tolist()


In [22]:
#cat_acc_feat

Reread data with objects turned into categories to save RAM:

In [23]:
dtype_acc = {}
for obj in cat_acc_feat:
    dtype_acc[obj] = 'category'
dtype_acc['policy_code'] = 'category'

dtype_rej = {}
for obj in cat_rej_feat:
    dtype_rej[obj] = 'category'
dtype_acc['Policy Code'] = 'category'

In [24]:
dtype_acc

{'id': 'category',
 'term': 'category',
 'grade': 'category',
 'sub_grade': 'category',
 'emp_title': 'category',
 'emp_length': 'category',
 'home_ownership': 'category',
 'verification_status': 'category',
 'issue_d': 'category',
 'loan_status': 'category',
 'pymnt_plan': 'category',
 'url': 'category',
 'desc': 'category',
 'purpose': 'category',
 'title': 'category',
 'zip_code': 'category',
 'addr_state': 'category',
 'earliest_cr_line': 'category',
 'initial_list_status': 'category',
 'last_pymnt_d': 'category',
 'next_pymnt_d': 'category',
 'last_credit_pull_d': 'category',
 'policy_code': 'category',
 'application_type': 'category',
 'verification_status_joint': 'category',
 'sec_app_earliest_cr_line': 'category',
 'hardship_flag': 'category',
 'hardship_type': 'category',
 'hardship_reason': 'category',
 'hardship_status': 'category',
 'hardship_start_date': 'category',
 'hardship_end_date': 'category',
 'payment_plan_start_date': 'category',
 'hardship_loan_status': 'category

In [25]:
LCacc_dd = dd.read_csv(csv_accepted, dtype=dtype_acc)
LCrej_dd = dd.read_csv(csv_rejected, dtype=dtype_rej)

In [26]:
LCacc_dd.npartitions

26

In [27]:
LCrej_dd.npartitions

27

In [28]:
LCacc_dd.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=68407277,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=68355089,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300

In [29]:
LCrej_dd.head()

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0.0


In [30]:
size_acc = [LCacc_dd.shape[0].compute(),  LCacc_dd.shape[1]]
size_rej = [LCrej_dd.shape[0].compute(),  LCrej_dd.shape[1]]

print('The "accepted" dataset has', size_acc[0], 'rows and', size_acc[1], 'features.')
print('The "rejected" dataset has', size_rej[0], 'rows and', size_rej[1], 'features.')

The "accepted" dataset has 2260701 rows and 151 features.
The "rejected" dataset has 27648741 rows and 9 features.


# 2. Data Cleaning

* Handling missing values.
* Removing duplicate samples and features.
* Remove unneccessary columns/rows.
* Treating (here rather checking) the outliers.

## Check Missing Values and Duplicates

Overview of amounts of Nan and of data type:

In [31]:
%reload_ext autoreload

lfunc.count_dtypes(LCacc_dd, name = 'LCacc')
lfunc.count_dtypes(LCrej_dd, name = 'LCrej')


The dataset LCacc has:
112 features of type float64.
39 features of type category.

The dataset LCrej has:
7 features of type category.
2 features of type float64.


In [32]:
%%time
%reload_ext autoreload

nan_overview_acc = lfunc.nan_type_overview_dd(LCacc_dd, size_acc[0])
nan_overview_acc.round(1).style.background_gradient(cmap="Blues")

CPU times: user 1min 38s, sys: 6.6 s, total: 1min 45s
Wall time: 1min 17s


,type,NaN[abs],NaN[%]
id,category,0,0.000000
member_id,float64,2260701,100.000000
loan_amnt,float64,33,0.000000
funded_amnt,float64,33,0.000000
funded_amnt_inv,float64,33,0.000000
term,category,33,0.000000
int_rate,float64,33,0.000000
installment,float64,33,0.000000
grade,category,33,0.000000
sub_grade,category,33,0.000000


In [33]:
%%time
%reload_ext autoreload

nan_overview_rej = lfunc.nan_type_overview_dd(LCrej_dd, size_rej[0])
nan_overview_rej

CPU times: user 56.6 s, sys: 4.48 s, total: 1min 1s
Wall time: 46.2 s


,type,NaN[abs],NaN[%]
Amount Requested,float64,0,0.0
Application Date,category,0,0.0
Loan Title,category,1303,0.0
Risk_Score,float64,18497630,66.9
Debt-To-Income Ratio,category,0,0.0
Zip Code,category,293,0.0
State,category,22,0.0
Employment Length,category,951355,3.4
Policy Code,category,918,0.0


### Drop Missing Value Columns

Inspect the frequently occuring number of missing values of 33. Take a feature that has this property:

In [34]:
%time
%reload_ext autoreload

LCacc_minnan_dd = LCacc_dd.loc[LCacc_dd.loan_amnt.isna()]
nan_overview_acc_minnan = lfunc.nan_type_overview_dd(LCacc_minnan_dd)
nan_overview_acc_minnan

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


,type,NaN[abs],NaN[%]
id,category,0.0,0.0
member_id,float64,33.0,100.0
loan_amnt,float64,33.0,100.0
funded_amnt,float64,33.0,100.0
funded_amnt_inv,float64,33.0,100.0
term,category,33.0,100.0
int_rate,float64,33.0,100.0
installment,float64,33.0,100.0
grade,category,33.0,100.0
sub_grade,category,33.0,100.0


In [35]:
del LCacc_minnan_dd  # free memory

Rows with a missing value in 'loan_amnt' have a missing value in every other feature.
Delete these rows:

In [36]:
LCacc_dd = LCacc_dd.dropna(subset = ['loan_amnt'])

Any hope for investigating the high rate missing value features?

In [37]:
df = LCacc_dd

df[~df.isnull().any(axis=1)].head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term


No chance to get a robust amount of rows without missing values!

Drop all columns with too many missing values:

In [38]:
cut_off_percentage = 70.

too_many_nan_acc = nan_overview_acc['NaN[%]'
                ].loc[nan_overview_acc['NaN[%]']
                > cut_off_percentage].round(1)

len(too_many_nan_acc), too_many_nan_acc.index.to_list()

(41,
 ['member_id',
  'desc',
  'mths_since_last_record',
  'mths_since_last_major_derog',
  'annual_inc_joint',
  'dti_joint',
  'verification_status_joint',
  'mths_since_recent_bc_dlq',
  'revol_bal_joint',
  'sec_app_fico_range_low',
  'sec_app_fico_range_high',
  'sec_app_earliest_cr_line',
  'sec_app_inq_last_6mths',
  'sec_app_mort_acc',
  'sec_app_open_acc',
  'sec_app_revol_util',
  'sec_app_open_act_il',
  'sec_app_num_rev_accts',
  'sec_app_chargeoff_within_12_mths',
  'sec_app_collections_12_mths_ex_med',
  'sec_app_mths_since_last_major_derog',
  'hardship_type',
  'hardship_reason',
  'hardship_status',
  'deferral_term',
  'hardship_amount',
  'hardship_start_date',
  'hardship_end_date',
  'payment_plan_start_date',
  'hardship_length',
  'hardship_dpd',
  'hardship_loan_status',
  'orig_projected_additional_accrued_interest',
  'hardship_payoff_balance_amount',
  'hardship_last_payment_amount',
  'debt_settlement_flag_date',
  'settlement_status',
  'settlement_date',


In [39]:
LCacc_dd = LCacc_dd.drop(columns=too_many_nan_acc.index.to_list())


In [40]:
%time
%reload_ext autoreload

nan_overview_acc_drop = lfunc.nan_type_overview_dd(LCacc_dd, size_acc[0])
nan_overview_acc_drop.round(1).style.background_gradient(cmap="Blues")

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs


,type,NaN[abs],NaN[%]
id,category,0,0.000000
loan_amnt,float64,0,0.000000
funded_amnt,float64,0,0.000000
funded_amnt_inv,float64,0,0.000000
term,category,0,0.000000
int_rate,float64,0,0.000000
installment,float64,0,0.000000
grade,category,0,0.000000
sub_grade,category,0,0.000000
emp_title,category,166969,7.400000


### Duplicates Check

In [41]:
%reload_ext autoreload

LCacc_dup = lfunc.get_dup_dd(LCacc_dd, name='LCacc', size=size_acc[0])
LCrej_dup = lfunc.get_dup_dd(LCrej_dd, name='LCrej', size=size_rej[0])

Total number of duplicates in " LCacc " : 0 ( 0.0 %).
Total number of duplicates in " LCrej " : 157612 ( 0.0 %).


Drop duplicated and repartition according to old setup:

In [42]:
n_part = LCrej_dd.npartitions

In [43]:
LCrej_dd = LCrej_dd.drop_duplicates(split_out = n_part)

In [44]:
LCrej_dd.npartitions

27

Verify:

In [45]:
%reload_ext autoreload

LCrej_dup = lfunc.get_dup_dd(LCrej_dd, name='LCrej', size=size_rej[0])

Total number of duplicates in " LCrej " : 0 ( 0.0 %).


In [46]:
%memit LCrej_dup
%memit LCacc_dup

peak memory: 1389.28 MiB, increment: 0.07 MiB
peak memory: 1389.73 MiB, increment: 0.20 MiB


In [47]:
del LCacc_dup, LCrej_dup

In [48]:
#%reload_ext autoreload

#lfunc.get_var_size(locals())

## Drops based on Statistical Overview

### Accepted Stats

In [49]:
LCacc_dd.describe().compute().T.round(1)

,count,mean,std,min,25%,50%,75%,max
loan_amnt,2260668.0,15046.9,9190.2,500.0,9000.0,14500.0,22000.0,40000.0
funded_amnt,2260668.0,15041.7,9188.4,500.0,9000.0,14500.0,22000.0,40000.0
funded_amnt_inv,2260668.0,15023.4,9192.3,0.0,9000.0,14500.0,22000.0,40000.0
int_rate,2260668.0,13.1,4.8,5.3,11.6,14.3,17.6,31.0
installment,2260668.0,445.8,267.2,4.9,278.5,410.7,634.8,1719.8
annual_inc,2260664.0,77992.4,112696.2,0.0,50000.0,70000.0,99985.2,110000000.0
dti,2258957.0,18.8,14.2,-1.0,12.6,18.8,25.6,999.0
delinq_2yrs,2260639.0,0.3,0.9,0.0,0.0,0.0,0.0,58.0
fico_range_low,2260668.0,698.6,33.0,610.0,680.0,700.0,735.0,845.0
fico_range_high,2260668.0,702.6,33.0,614.0,684.0,704.0,739.0,850.0


In [50]:
LCacc_dd_obj = LCacc_dd.describe(exclude=np.number).compute().T
LCacc_dd_obj['freq[%]'] = (LCacc_dd_obj['freq'].astype('float')/LCacc_dd_obj['count'].astype('float')*100)
LCacc_dd_obj.round(1)

,unique,count,top,freq,freq[%]
id,2260668,2260668,361774,1,0.0
term,2,2260668,36 months,1609754,71.2
grade,7,2260668,B,663557,29.4
sub_grade,35,2260668,C1,145903,6.5
emp_title,512694,2093699,Teacher,38824,1.9
emp_length,11,2113761,10+ years,748005,35.4
home_ownership,6,2260668,MORTGAGE,1111450,49.2
verification_status,3,2260668,Source Verified,886231,39.2
issue_d,139,2260668,Mar-2016,61992,2.7
loan_status,9,2260668,Fully Paid,1076751,47.6


In [51]:
%memit LCacc_dd_obj

peak memory: 1587.89 MiB, increment: -1.99 MiB


In [52]:
cut_off_percentage = 95.

extreme_majority_class_acc = LCacc_dd_obj['freq[%]'
                ].loc[LCacc_dd_obj['freq[%]']
                > cut_off_percentage].index.to_list()

len(extreme_majority_class_acc), extreme_majority_class_acc

(6,
 ['pymnt_plan',
  'next_pymnt_d',
  'policy_code',
  'hardship_flag',
  'disbursement_method',
  'debt_settlement_flag'])

In [53]:
cut_off = 1000.

too_many_unique_acc = LCacc_dd_obj['unique'
                ].loc[LCacc_dd_obj['unique'].astype('float')
                > cut_off].index.to_list()

len(too_many_unique_acc), too_many_unique_acc

(4, ['id', 'emp_title', 'url', 'title'])

In [54]:
cut_off = 0.05 * size_acc[0]

too_few_entries_acc = LCacc_dd_obj['count'
                ].loc[LCacc_dd_obj['count'].astype('float')
                < cut_off].index.to_list()

len(too_few_entries_acc), too_few_entries_acc

(0, [])

### Rejected Stats

In [55]:
LCrej_dd.describe().compute().T.round(1)

,count,mean,std,min,25%,50%,75%,max
Amount Requested,27490787.0,13104.1,14887.8,0.0,5000.0,10000.0,20000.0,1400000.0
Risk_Score,9130929.0,628.2,89.8,0.0,592.0,637.0,675.0,990.0


In [56]:
LCrej_dd_obj = LCrej_dd.describe(exclude=np.number).compute().T
LCrej_dd_obj['freq[%]'] = (LCrej_dd_obj['freq'].astype('float')/LCrej_dd_obj['count'].astype('float')*100)
LCrej_dd_obj.round(1)

,unique,count,top,freq,freq[%]
Application Date,4238,27490787,2018-12-04,41696,0.2
Loan Title,73928,27489504,Debt consolidation,6362745,23.1
Debt-To-Income Ratio,126145,27490787,100%,1311209,4.8
Zip Code,1001,27490495,112xx,262986,1.0
State,51,27490765,CA,3218415,11.7
Employment Length,11,26541085,< 1 year,22841895,86.1
Policy Code,2,27489869,0.0,27401835,99.7


In [57]:
cut_off_percentage = 95.

extreme_majority_class_rej = LCrej_dd_obj['freq[%]'
                ].loc[LCrej_dd_obj['freq[%]']
                > cut_off_percentage].index.to_list()

len(extreme_majority_class_rej), extreme_majority_class_rej

(1, ['Policy Code'])

In [58]:
cut_off = 1010.

too_many_unique_rej = LCrej_dd_obj['unique'
                ].loc[LCrej_dd_obj['unique'].astype('float')
                > cut_off].index.to_list()

len(too_many_unique_rej), too_many_unique_rej

(3, ['Application Date', 'Loan Title', 'Debt-To-Income Ratio'])

In [59]:
cut_off = 0.05 * size_rej[0]

too_few_entries_rej = LCrej_dd_obj['count'
                ].loc[LCrej_dd_obj['count'].astype('float')
                < cut_off].index.to_list()

len(too_few_entries_rej), too_few_entries_rej

(0, [])

### Other Checks and Modifications

Columns that contain too complex, duplicate or irrelevant information:

In [60]:
to_drop_acc = []
to_drop_rej = []

#### Accepted

**not known before loan acceptance/grade/interest estimate**:

* repayment progress (i.e. after loan acceptance): installment (depends on int_rate, loan amount, term), funded_amnt, funded_amnt_inv,
out_prncp, 	out_prncp_inv, 	total_pymnt, 	total_pymnt_inv, 	total_rec_prncp, 	total_rec_int, 	total_rec_late_fee, last_pymnt_d, 	last_pymnt_amnt, 	last_credit_pull_d, 	last_fico_range_high, next_pymnt_d, collection_recovery_fee, recoveries


**not needed for loan acceptance/grade/interest estimate**:

* id, member_id, emp_title, url, title, (all done above)

* clarify: verification_status, pymnt plan, (pymnt plan: done above)

* if application_type = individual only: drop
annual_inc_joint, dti_joint, verification_status_joint, application_type --> lose 5.4% of the rows

* initial_list_status



**duplicate**:
* funded_amnt = loan_amnt, except from 0.1% of the cases
* (last_)fico_range_low = ((last_)fico_range_high - 4) in most cases




**dates to years**:
* issued_d
* earliest_cr_line


In [61]:
to_drop_acc.append('initial_list_status')

##### Repayment variables

In [62]:
repay = ['installment',
         'funded_amnt',
         'funded_amnt_inv',
         'out_prncp',
         'out_prncp_inv',
         'total_pymnt',
         'total_pymnt_inv',
         'total_rec_prncp',
         'total_rec_int',
         'total_rec_late_fee',
         'last_pymnt_d',
         'last_pymnt_amnt',
         'last_credit_pull_d',
         'last_fico_range_high',
         'next_pymnt_d',
         'collection_recovery_fee',
         'recoveries'
        ]

##### App Type

In [63]:
LCacc_dd.application_type.value_counts().compute()


,application_type
Individual,2139958
Joint App,120710


Drop Joint applications:

In [64]:
LCacc_dd = LCacc_dd.loc[LCacc_dd.application_type=='Individual']

In [65]:
to_drop_acc.append('application_type')

##### funded_amnt

Reduncancy in loan and funded amount:

In [66]:
LCacc_dd_diff = LCacc_dd.loc[~(LCacc_dd.loan_amnt == LCacc_dd.funded_amnt)]

In [67]:
len(LCacc_dd_diff)

2064

That's 0.1% of the data - ignore/drop.

In [68]:
LCacc_dd_diff_inv = LCacc_dd.loc[~(LCacc_dd.loan_amnt == LCacc_dd.funded_amnt_inv)]

In [69]:
len(LCacc_dd_diff_inv)

146788

That are about 7.5% of the data -> keep.

---> Drop 'funded_amnt'

In [70]:
to_drop_acc.append('funded_amnt')

##### FICO

Reduncancy in FICO high/low values:

In [71]:
(LCacc_dd.fico_range_high - LCacc_dd.fico_range_low).value_counts().compute()

,0
4.0,2139671
5.0,287


Difference between current high/low FICO value is 4 - 5. Drop one of them.


In [72]:
to_drop_acc.append('fico_range_low')

In [73]:
(LCacc_dd.last_fico_range_high - LCacc_dd.last_fico_range_low).value_counts().compute()

,0
4.0,2102972
499.0,36418
5.0,334
0.0,234


Difference between last high/low FICO value is 4-5 for most of the data, except from about 37300 (1.6%) rows. Drop one of them.


In [74]:
to_drop_acc.append('last_fico_range_low')

In [75]:
((LCacc_dd.last_fico_range_high - LCacc_dd.fico_range_high).value_counts().compute())[:20]

,0
0.0,104962
5.0,98920
10.0,95614
-5.0,91438
15.0,90030
20.0,82786


Differences between last and current high FICO vary drastically (in increments of 5) - perhaps we will need both features.

In [76]:
#LCacc_dd.loc[LCacc_dd.dti < 0.01].sort_values(by = 'dti', ascending=False).head()  # .describe() shows min(dti) = -1 --- where is it???

##### dates to years

In [77]:
LCacc_dd_issue_dt = dd.to_datetime(LCacc_dd['issue_d'])
LCacc_dd['issue_d_year'] = LCacc_dd_issue_dt.dt.year
LCacc_dd['issue_d_month'] = LCacc_dd_issue_dt.dt.month

In [78]:
to_drop_acc.extend(['issue_d_month', 'issue_d'])


In [79]:
LCacc_dd_early_cr_dt = dd.to_datetime(LCacc_dd['earliest_cr_line'])
LCacc_dd['earliest_cr_line_year'] = LCacc_dd_early_cr_dt.dt.year
LCacc_dd['earliest_cr_line_month'] = LCacc_dd_early_cr_dt.dt.month

In [80]:
to_drop_acc.extend(['earliest_cr_line_month', 'earliest_cr_line'])

##### modify Loan_status

In [81]:
original = 'loan_status'
simple = original+'_simple'

simple_loan_status = {
'Does not meet the credit policy. Status:Fully Paid': 'Fully Paid',
'Does not meet the credit policy. Status:Charged Off': 'Charged Off',
'Default': 'Charged Off',
'In Grace Period': 'Delayed',
'Late (16-30 days)': 'Delayed',
'Late (31-120 days)': 'Delayed'
}

LCacc_dd[original] = LCacc_dd[original].astype('object')
LCacc_dd[simple] = LCacc_dd[original]
for key, value in zip(simple_loan_status.keys(), simple_loan_status.values()):
    #LCacc_dd.loc[LCacc_dd[original] == key, simple] = value
    LCacc_dd[simple] = LCacc_dd[simple].mask(LCacc_dd[original] == key, value)
LCacc_dd[simple] = LCacc_dd[simple].astype('category')
LCacc_dd[original] = LCacc_dd[original].astype('category')


#### Rejected

##### dates to years

In [82]:
LCrej_dd_appl_dt = dd.to_datetime(LCrej_dd['Application Date'])
LCrej_dd['applied_year'] = LCrej_dd_appl_dt.dt.year
LCrej_dd['applied_month'] = LCrej_dd_appl_dt.dt.month

In [83]:
to_drop_rej.extend(['applied_month', 'Application Date'])

##### dti

In [84]:
%reload_ext autoreload

LCrej_dd['dti'] = LCrej_dd['Debt-To-Income Ratio'].apply(lambda x: lfunc.replace_with_nan(x, '%', '')).astype('float')

In [85]:
to_drop_rej.append('Debt-To-Income Ratio')

In [86]:
LCrej_dd.npartitions

27

### Drops

#### Drops for accepted

In [87]:
to_drop_all_acc = to_drop_acc + repay + extreme_majority_class_acc \
+ too_many_unique_acc + too_few_entries_acc

In [88]:
print('Additional columns to drop besides NaN and duplicates:')
to_drop_all_acc

Additional columns to drop besides NaN and duplicates:


['initial_list_status',
 'application_type',
 'funded_amnt',
 'fico_range_low',
 'last_fico_range_low',
 'issue_d_month',
 'issue_d',
 'earliest_cr_line_month',
 'earliest_cr_line',
 'installment',
 'funded_amnt',
 'funded_amnt_inv',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'last_credit_pull_d',
 'last_fico_range_high',
 'next_pymnt_d',
 'collection_recovery_fee',
 'recoveries',
 'pymnt_plan',
 'next_pymnt_d',
 'policy_code',
 'hardship_flag',
 'disbursement_method',
 'debt_settlement_flag',
 'id',
 'emp_title',
 'url',
 'title']

In [89]:
LCacc_dd = LCacc_dd.drop(columns=to_drop_all_acc)

In [90]:
print('Number of features reduced to:', LCacc_dd.shape[1])

Number of features reduced to: 81


In [91]:
LCrej_dd.npartitions

27

#### Drops for rejected

In [92]:
to_drop_all_rej = to_drop_rej + extreme_majority_class_rej \
+ too_many_unique_rej + too_few_entries_rej

In [93]:
print('Additional columns to drop besides NaN and duplicates:')
to_drop_all_rej

Additional columns to drop besides NaN and duplicates:


['applied_month',
 'Application Date',
 'Debt-To-Income Ratio',
 'Policy Code',
 'Application Date',
 'Loan Title',
 'Debt-To-Income Ratio']

In [94]:
LCrej_dd = LCrej_dd.drop(columns=to_drop_all_rej)

In [95]:
print('Number of features reduced to:', LCrej_dd.shape[1])

Number of features reduced to: 7


In [96]:
LCrej_dd.columns

Index(['Amount Requested', 'Risk_Score', 'Zip Code', 'State',
       'Employment Length', 'applied_year', 'dti'],
      dtype='object')

In [97]:
LCrej_dd.npartitions

27

## Export

In [98]:
parquet_home_acc_cleaned = home_folder+'parquet_LCacc_cleaned/'
parquet_home_rej_cleaned = home_folder+'parquet_LCrej_cleaned/'

if not os.path.exists(parquet_home_acc_cleaned):
    os.mkdir(parquet_home_acc_cleaned)
if not os.path.exists(parquet_home_rej_cleaned):
    os.mkdir(parquet_home_rej_cleaned)


In [99]:
# home_folder = '/content/gdrive/MyDrive/Colab Notebooks/Turing_Module_Sprints/M3-S3_LendingClub/'

In [100]:
### add to parquet write: name_function=lambda i: f"data-{i}.parquet

In [101]:
LCacc_dd.to_parquet(parquet_home_acc_cleaned, engine="fastparquet")#schema=None)

In [102]:
LCrej_dd.to_parquet(parquet_home_rej_cleaned, engine="fastparquet")#schema=None)

In [103]:
#%%capture
#LCacc_dd.to_csv(home_folder+'M3-S3-LendingClub_acc_cleaned/part*.csv', index=False)

In [104]:
#%%capture
#LCrej_dd.to_csv(home_folder+'M3-S3-LendingClub_rej_cleaned/part*.csv', index=False)

## Sampling

#### Accepted Sampling

In [105]:
accSAMPLES = 10000
frac_acc = accSAMPLES/size_acc[0]

LCacc_dd_EDA=LCacc_dd.sample(frac=accSAMPLES/size_acc[0], replace=None, random_state=10)

In [106]:
LCacc_EDA = LCacc_dd_EDA.compute()
len(LCacc_EDA)

9466

In [107]:
if not os.path.exists(home_folder+'cleaned/'):
    os.mkdir(home_folder+'cleaned/')

In [108]:
LCacc_EDA.to_csv(home_folder+'cleaned/LendingClub_EDA_acc.csv', index=False)

#### Rejected sampling

In [109]:
rejSAMPLES = 10000

#LCrej_dd_s=LCrej_dd.sample(frac=frac_acc, replace=None, random_state=10)
LCrej_dd__EDA=LCrej_dd.sample(frac=rejSAMPLES/size_rej[0], replace=None, random_state=10)

In [110]:
LCrej_EDA = LCrej_dd__EDA.compute()
len(LCrej_EDA)

9942

In [111]:
LCrej_EDA.to_csv(home_folder+'cleaned/LendingClub_EDA_rej.csv', index=False)